In [1]:
from csv import reader
from sklearn.tree import DecisionTreeClassifier
from sklearn.cross_validation import train_test_split
from sklearn import preprocessing
from sklearn import tree
from sklearn.externals.six import StringIO
import calendar

In [2]:
fileValues = []

In [3]:
def readInValuesFromCSV():
    # open csv file and append to a list of dictionaries
    infile = open("NYPD_7_Major_Felony_Incidents.csv", 'r')
    first = True
    for line in reader( infile ):
#         print line
        if not first:
            OBJECTID,Identifier,Occurrence_Date,Day_of_Week,Occurrence_Month,Occurrence_Day,Occurrence_Year,Occurrence_Hour,CompStat_Month,CompStat_Day,CompStat_Year,Offense,Offense_Classification,Sector,Precinct,Borough,Jurisdiction,XCoordinate,YCoordinate,Location_1 = line
            
            values = {'OBJECTID' : OBJECTID,'Identifier':Identifier,'Occurrence Date':Occurrence_Date,'Day of Week':Day_of_Week,'Occurrence Month':Occurrence_Month,'Occurrence Day':Occurrence_Day,'Occurrence Year':Occurrence_Year,'Occurrence Hour':Occurrence_Hour,'CompStat Month':CompStat_Month,'CompStat Day':CompStat_Day,'CompStat Year':CompStat_Year,'Offense':Offense,'Offense Classification':Offense_Classification,'Sector':Sector,'Precinct':Precinct,'Borough':Borough,'Jurisdiction':Jurisdiction,'XCoordinate':XCoordinate,'YCoordinate':YCoordinate,'Location 1':Location_1 }
            
            fileValues.append(values)
            
        first = False

In [4]:
readInValuesFromCSV()

In [6]:
fileValues[2]

{'Borough': 'BROOKLYN',
 'CompStat Day': '31',
 'CompStat Month': '10',
 'CompStat Year': '2008',
 'Day of Week': 'Friday',
 'Identifier': '716dbc6f',
 'Jurisdiction': 'N.Y. POLICE DEPT',
 'Location 1': '(40.688874254, -73.9918594329999)',
 'OBJECTID': '3',
 'Occurrence Date': '10/30/1970 03:30:00 PM',
 'Occurrence Day': '30',
 'Occurrence Hour': '15',
 'Occurrence Month': 'Oct',
 'Occurrence Year': '1970',
 'Offense': 'BURGLARY',
 'Offense Classification': 'FELONY',
 'Precinct': '84',
 'Sector': 'H',
 'XCoordinate': '986508',
 'YCoordinate': '190249'}

In [7]:
precinctList = []
for precinct in fileValues:
    precinctList.append(precinct['Precinct'])

In [8]:
set(precinctList)

{'1',
 '10',
 '100',
 '101',
 '102',
 '103',
 '104',
 '105',
 '106',
 '107',
 '108',
 '109',
 '110',
 '111',
 '112',
 '113',
 '114',
 '115',
 '120',
 '121',
 '122',
 '123',
 '13',
 '14',
 '17',
 '18',
 '19',
 '20',
 '22',
 '23',
 '24',
 '25',
 '26',
 '28',
 '30',
 '32',
 '33',
 '34',
 '40',
 '41',
 '42',
 '43',
 '44',
 '45',
 '46',
 '47',
 '48',
 '49',
 '5',
 '50',
 '52',
 '6',
 '60',
 '61',
 '62',
 '63',
 '66',
 '67',
 '68',
 '69',
 '7',
 '70',
 '71',
 '72',
 '73',
 '75',
 '76',
 '77',
 '78',
 '79',
 '81',
 '83',
 '84',
 '88',
 '9',
 '90',
 '94',
 'NA'}

In [9]:
# make sure that there are no empty values
crimeData = []
for crime in fileValues:
    if crime['Precinct'] != "" and crime['Offense'] != "" and crime['Day of Week'] != "" and crime['Occurrence Hour'] != "" and crime['Jurisdiction'] == "N.Y. POLICE DEPT":
        crimeData.append(crime)

In [10]:
# encode precints and offense into floats
offenseEncoder = preprocessing.LabelEncoder()
precinctEncoder = preprocessing.LabelEncoder()

In [11]:
offenseEncoder.fit([crime['Offense'] for crime in crimeData])
precinctEncoder.fit([crime['Precinct'] for crime in crimeData])

LabelEncoder()

In [30]:
joblib.dump(offenseEncoder, 'offenseEncoder.pkl', compress=9)
joblib.dump(precinctEncoder, 'precinctEncoder.pkl', compress=9)

['precinctEncoder.pkl']

In [12]:
#split data into training data and test data
X_train, X_test, y_train, y_test = train_test_split(precinctEncoder.transform([x['Precinct'] for x in crimeData]),offenseEncoder.transform([x['Offense'] for x in crimeData]),test_size=0.10)

In [13]:
# init the decisionTreeClassifier and fit the training data
dt = DecisionTreeClassifier(random_state=0)
dt.fit(X_train.reshape(-1,1),y_train.reshape(-1,1))

DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
            max_features=None, max_leaf_nodes=None, min_samples_leaf=1,
            min_samples_split=2, min_weight_fraction_leaf=0.0,
            random_state=0, splitter='best')

## Prediction Score

In [21]:
dt.score(X_test.reshape(-1,1),y_test.reshape(-1,1))

0.3897405830393787

In [23]:
from sklearn.externals import joblib
joblib.dump(dt, 'dt_model.pkl', compress=9)

['dt_model.pkl']

## Predict a crime

In [15]:
print offenseEncoder.inverse_transform(
    dt.predict(
        [precinctEncoder.transform(
                ['94'])]))

['GRAND LARCENY']


In [16]:
def predictCrimeInPrecinct(precinctNumber):
    print offenseEncoder.inverse_transform(
    dt.predict(
        [precinctEncoder.transform(
                [precinctNumber])]))

In [17]:
predictCrimeInPrecinct(84)

['GRAND LARCENY']


In [18]:
uniquePrecincts = set(precinctList)

In [32]:
len(uniquePrecincts)

78

In [19]:
for precinct in uniquePrecincts:
    print(precinct)
    predictCrimeInPrecinct(precinct)

77
['GRAND LARCENY']
66
['GRAND LARCENY']
42
['ROBBERY']
62
['GRAND LARCENY']
112
['GRAND LARCENY']
22
['GRAND LARCENY']
50
['GRAND LARCENY']
115
['GRAND LARCENY']
61
['GRAND LARCENY']
88
['GRAND LARCENY']
63
['GRAND LARCENY']
111
['GRAND LARCENY']
110
['GRAND LARCENY']
113
['GRAND LARCENY']
67
['GRAND LARCENY']
68
['GRAND LARCENY']
83
['BURGLARY']
52
['GRAND LARCENY']
84
['GRAND LARCENY']
23
['GRAND LARCENY']
24
['GRAND LARCENY']
25
['GRAND LARCENY']
26
['GRAND LARCENY']
81
['GRAND LARCENY']
20
['GRAND LARCENY']
48
['ROBBERY']
49
['GRAND LARCENY']
46
['FELONY ASSAULT']
47
['FELONY ASSAULT']
44
['FELONY ASSAULT']
45
['GRAND LARCENY']
28
['GRAND LARCENY']
43
['GRAND LARCENY']
40
['GRAND LARCENY']
41
['FELONY ASSAULT']
1
['GRAND LARCENY']
5
['GRAND LARCENY']
79
['GRAND LARCENY']
7
['GRAND LARCENY']
6
['GRAND LARCENY']
9
['GRAND LARCENY']
78
['GRAND LARCENY']
122
['GRAND LARCENY']
10
['GRAND LARCENY']
NA
['MURDER & NON-NEGL. MANSLAUGHTE']
120
['GRAND LARCENY']
76
['GRAND LARCENY']
108
['G

In [20]:
crimeOffenses = []
for crime in fileValues:

IndentationError: expected an indented block (<ipython-input-20-58d7d78ad675>, line 2)